# Event handlers

This example demonstrate how Unfolded Map SDK makes it possible to register event handlers to receive notifications for specific map events.

## Creating a map

In [57]:
from unfolded.map_sdk import UnfoldedMap
unfolded_map = UnfoldedMap()

In [58]:
from sidecar import Sidecar
sc = Sidecar(title='Earthquakes', anchor='split-right')
with sc:
    display(unfolded_map)

## Data loading

In [59]:
import pandas as pd
df = pd.read_csv('./data/earthquakes.csv')

## Adding the data to the map

In [60]:
unfolded_map.add_dataset({
    'label': 'Earthquakes',
    'data': df
})

<Future pending>

## Adding a filter

In [8]:
magnitude_extent = [df['Magnitude'].min(), df['Magnitude'].max()]

unfolded_map.set_filter({
    'id': 'magnitude_filter',
    'field': 'Magnitude',
    'value': [5, magnitude_extent[1]]
})

## Controlling the filter with a range slider

In [9]:
import ipywidgets as widgets

slider = widgets.FloatRangeSlider(
    value=magnitude_extent,
    min=magnitude_extent[0],
    max=magnitude_extent[1],
    step=0.1,
    description='Magnitude:',
    continuous_update=True
)

In [47]:
def update_value_filter(change):
    if 'new' in change and 'value' in change['new']:
        unfolded_map.set_filter({
            'id': 'magnitude_filter',
            'field': 'Magnitude',
            'value': change['new']['value']
        })

In [48]:
slider.observe(update_value_filter)
display(slider)

FloatRangeSlider(value=(3.9, 5.9), description='Magnitude:', max=7.39, min=2.5)

The following will unregister the observer:

In [45]:
slider.unobserve(None)

## Listening to filter change events

You can register event handlers to be notified of filter changes in the map. Here's how we can syncronize the above range slider with the Magnitude filter in the map:

In [52]:
def on_filter_sync(info):
    if 'Magnitude' in info['name'] and info['prop'] == 'value':
        slider.value = info['value']

unfolded_map.set_map_event_handlers({
    'on_filter': on_filter_sync
})

Now try changing the "Magnitude" filter in the "Filter" pane of the left sidebar in the map. You should see the slider above in the notebook update. 

##  Debugging a filter event handler

Here's how you can output the event info when filter events are triggered:

In [20]:
import ipywidgets as widgets
output = widgets.Output()
@output.capture(clear_output=True)
def on_filter_output(info):
    print(info)
output

Output()

In [36]:
unfolded_map.set_map_event_handlers({
    'on_filter': on_filter_output
})